<font color="green">*To start working on this notebook, or any other notebook that we will use in the Moringa Data Science Course, we will need to save our own copy of it. We can do this by clicking File > Save a Copy in Drive. We will then be able to make edits to our own copy of this notebook.*</font>

# Python Programming: Ridge Regression

## 1.0 Example 

In [1]:
# Example 
# ---
# Regularization is the process of penalizing coefficients of variables either by removing them and or reducing their impact. 
# Ridge regression reduces the effect of problematic variables close to zero but never fully removes them. 
# ---
# Question: Build a regrssion model to predict expenses based on the variables available.
# ---
# Dataset source: Pydataset Library: VietNamI Dataset
# ---
#

In [2]:
# Importing our libraries
# 
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
# installing !pip install pydataset and importing pydataset so as to use a dataset from the package
# 
!pip install pydataset
from pydataset import data 

     |████████████████████████████████| 15.9 MB 486 kB/s 
  Created wheel for pydataset: filename=pydataset-0.2.0-py3-none-any.whl size=15939432 sha256=3fea787222291058dd2350b18fd58f66218ff3a45c4a49364f817db5471a8e24
  Stored in directory: /root/.cache/pip/wheels/32/26/30/d71562a19eed948eaada9a61b4d722fa358657a3bfb5d151e2
Successfully built pydataset
initiated datasets repo at: /root/.pydataset/


In [4]:
# Data Preparation
# 

# Loading the data and convert the sex variable to a dummy variable
#
df = pd.DataFrame(data('VietNamI'))

# Transforming categorical data to numeric
df.loc[df.sex== 'male','sex'] = 0
df.loc[df.sex== 'female','sex'] = 1
df['sex'] = df['sex'].astype(int)

# Setting up our X and y datasets
#
X = df[['pharvis','age','sex','married','educ','illness','injury','illdays','actdays','insurance']]
y = df['lnhhexp']

In [5]:
df.head()

,pharvis,lnhhexp,age,sex,married,educ,illness,injury,illdays,actdays,insurance,commune
1,0,2.730363,3.761200,0,1,2,1,0,7,0,0,192
2,0,2.737248,2.944439,1,0,0,1,0,4,0,0,167
3,0,2.266935,2.564950,0,0,4,0,0,0,0,1,76
4,1,2.392753,3.637586,1,1,3,1,0,3,0,1,123
5,1,3.105335,3.295837,0,1,3,1,0,10,0,0,148


In [6]:
# Creating our baseline regression model
# This is a model that has no regularization to it
# 
regression = LinearRegression()
regression.fit(X,y)
first_model = (mean_squared_error(y_true=y,y_pred=regression.predict(X)))
print(first_model)

# The output  value of 0.355289 will be our indicator to determine if the regularized ridge regression model is superior or not.

0.35528915032173053


In [7]:
# In order to create our ridge model we need to first determine the most appropriate value for the l2 regularization. 
# L2 is the name of the hyperparameter that is used in ridge regression. 
# Determining the value of a hyperparameter requires the use of a grid. 
# In the code below, we first create our ridge model and indicate normalization in order to get better estimates. 
# Next we setup the grid that we will use. 
# The search object has several arguments within it. Alpha is hyperparameter we are trying to set. 
# The log space is the range of values we want to test. 
# We want the log of -5 to 2, but we only get 8 values from within that range evenly spread out. 
# Are metric is the mean squared error. Refit set true means to adjust the parameters while modeling 
# and cv is the number of folds to develop for the cross-validation. 
#
ridge = Ridge(normalize=True)
search = GridSearchCV(estimator=ridge,param_grid={'alpha':np.logspace(-5,2,8)},scoring='neg_mean_squared_error',n_jobs=1,refit=True,cv=10)

In [8]:
# We now use the .fit function to run the model and then use the .best_params_ and
#  .best_scores_ function to determine the models strength. 
# 
search.fit(X,y)
search.best_params_
{'alpha': 0.01}
abs(search.best_score_) 

# The best_params_ tells us what to set alpha too which in this case is 0.01. 
# The best_score_ tells us what the best possible mean squared error is. 
# In this case, the value of 0.38 is worse than what the baseline model was. 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipel

0.38013256937541345

In [9]:
# We can confirm this by fitting our model with the ridge information and finding the mean squared error below
#
ridge = Ridge(normalize=True,alpha=0.01)
ridge.fit(X,y)
second_model = (mean_squared_error(y_true=y,y_pred=ridge.predict(X)))
print(second_model)

0.3552932199260658


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  FutureWarning,


In [10]:
# The 0.35 is lower than the 0.38. This is because the last results are not cross-validated. 
# In addition, these results indicate that there is little difference between the ridge and baseline models. 
# This is confirmed with the coefficients of each model found below.
# 
coef_dict_baseline = {}
for coef, feat in zip(regression.coef_,data("VietNamI").columns):
    coef_dict_baseline[feat] = coef
coef_dict_baseline

# The coefficient values are about the same. This means that the penalization made little difference with this dataset.

{'actdays': 0.14687843649771123,
 'age': 0.004012412278795818,
 'educ': -0.06180921300600299,
 'illdays': -0.0067170633108930965,
 'illness': 0.040870384578962464,
 'injury': -0.00276376871656904,
 'lnhhexp': 0.06480086550467884,
 'married': 0.07527646383836208,
 'pharvis': 0.01328205088695085,
 'sex': -0.08739614349708981}

## 2.0 Challenges

### <font color="green">Challenge 1</font>

In [11]:
# Challenge 1 
# ---
# Question: Build an accurate model that can estimate the weight of fish given the following dataset.
# ---
# Dataset url = http://bit.ly/FishDataset
# ---
url = 'http://bit.ly/FishDataset'
df = pd.read_csv(url)
df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [12]:
# Setting up our X and y datasets
#
X = df[['Length1','Length2','Length3','Height','Width']]
y = df['Weight']

In [13]:
# Creating our baseline regression model
# This is a model that has no regularization to it

regression = LinearRegression()
regression.fit(X,y)
first_model = (mean_squared_error(y_true=y,y_pred=regression.predict(X)))
print(first_model)

# The output  value of 14607.87 will be our indicator to determine if the regularized ridge regression model is superior or not.

14607.878944541946


In [14]:
y_pred=regression.predict(X)
comparison_frame = pd.DataFrame({'Actual': y, 'Predicted': y_pred})
comparison_frame

,Actual,Predicted
0,242.0,326.816128
1,290.0,369.578593
2,340.0,370.824180
3,363.0,439.056139
4,430.0,444.169168
5,450.0,466.125319
6,500.0,496.151343
7,390.0,473.623986
8,450.0,511.951893
9,500.0,540.445456


In [16]:
# Inserting Ridge regression
ridge = Ridge(normalize=True)
search = GridSearchCV(estimator=ridge,param_grid={'alpha':np.logspace(-5,2,8)},scoring='neg_mean_squared_error',n_jobs=1,refit=True,cv=10)
search.fit(X,y)
search.best_params_
{'alpha': 1.0}
abs(search.best_score_) 

# From the result, 22554.15 these indicate that there is little difference between the ridge and baseline models.

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipel

22554.158128353116

### <font color="green">Challenge 2</font>

In [ ]:
# Challenge 2
# ---
# Question: Build a regression algorithm for predicting unemployment within an economy.
# ---
# Dataset url = http://bit.ly/EconomicDataset
# ---
# Dataset Info
# 1. date. Month of data collection
# 2. psavert, personal savings rate
# 3. pce, personal consumption expenditures, in billions of dollars
# 4. unemploy, number of unemployed in thousands 
# 5. empmed, median duration of unemployment, in week
# 6. pop, total population, in thousands
# ---
# 
OUR CODE GOES HERE

### <font color="green">Challenge 3</font>

In [ ]:
# Challenge 3
# ---
# Question: Build a regression model to predict the life expectancy of a country. 
# Apply ridge regression to your model.
# ---
# Dataset url = http://bit.ly/LifeExpectancyDataset
# ---
# Dataset Info:
# Country: Country
# Year: Year
# Status: Developed or Developing status
# Life expectancy: Life Expectancy in age
# Adult Mortality: Adult Mortality Rates of both sexes (probability of dying between 15 and 60 years per 1000 population)
# infant deaths: Number of Infant Deaths per 1000 population
# Alcohol: Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)
# percentage expenditure: Expenditure on health as a percentage of Gross Domestic Product per capita(%)
# Hepatitis B: Hepatitis B (HepB) immunization coverage among 1-year-olds (%)
# Measles: Measles: number of reported cases per 1000 population
# BMI: Average Body Mass Index of entire population
# under-five: deaths Number of under-five deaths per 1000 population
# Polio: Polio (Pol3) immunization coverage among 1-year-olds (%)
# Total expenditure: General government expenditure on health as a percentage of total government expenditure (%)
# Diphtheria: Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%)
# HIV/AIDS: Deaths per 1 000 live births HIV/AIDS (0-4 years)
# GDP: Gross Domestic Product per capita (in USD)
# Population: Population of the country
# thinness 1-19 years: Prevalence of thinness among children and adolescents for Age 10 to 19 (% )
# thinness 5-9 years: Prevalence of thinness among children for Age 5 to 9(%)
# Income composition of resources: Human Development Index in terms of income composition of resources (index ranging from 0 to 1)
# Schooling: Number of years of Schooling(years)
# ---
# 
OUR CODE GOES HERE

### <font color="green">Challenge 4</font>

In [ ]:
# Challenge 4
# ---
# Question: Given the beauty dataset below, create a regression model to predict wages upon applying ridge regression.
# ---
# Dataset url = http://bit.ly/BeautyDataset
# ---
# 
OUR CODE GOES HERE

### <font color="green">Challenge 5</font>

In [ ]:
# Challenge 5
# ---
# Create a regression model to predict sales prices. 
# Apply regularization techniques.
# ---
# Dataset source = http://bit.ly/HousePricesDataset
# ---
# 
OUR CODE GOES HERE